In [ ]:
print((186*20*10)/60)

In [ ]:
from social_agents.agent_builder import SocialAgentBuilder
from social_agents.utils import eval_experiment
import pandas as pd
from glob import glob
import os
all_results_path = SocialAgentBuilder.ROOT_FOLDER + "experiment_results.csv"
path_ = SocialAgentBuilder.ROOT_FOLDER + "output_*.json"
time_log_path = SocialAgentBuilder.ROOT_FOLDER + "time_log.csv"

out_files = [x for x in glob(path_) if x.find("_eval_") == -1]
evaluated_files = [x for x in glob(path_) if x.find("_eval_") > -1]
data_split = "validation"
threshold = 0.6
metric = "similarity"


all_results = []
for out_ in out_files:
    eval_name = out_.replace("json", f"_eval_{metric}_{str(threshold).replace('.', '')}.json")
    if eval_name in evaluated_files:
        print("already evaluated")
        continue
    
    exp_name = os.path.basename(out_).replace("output_", "").replace(".json", "")
    eval_dict = {"experiment_name": exp_name}
    eval_dict = eval_dict | eval_experiment(submission_path=out_, data_split=data_split, threshold=threshold)

    time_log_df = pd.read_csv(time_log_path)
    if exp_name not in time_log_df.columns:
        print(f"time not logged for {exp_name}")
    else:
        eval_dict["time_mean"] = time_log_df[exp_name].mean()
        eval_dict["time_std"] = time_log_df[exp_name].std()
    try:
        all_experiments_results_df = pd.read_csv(all_results_path)
        
        all_experiments_results_df = all_experiments_results_df.drop(columns=[col for col in all_experiments_results_df.columns if col.startswith('Unnamed')])
    except FileNotFoundError:
        all_experiments_results_df = pd.DataFrame()
    

    all_experiments_results_df = pd.concat([all_experiments_results_df, pd.DataFrame([eval_dict])], ignore_index=True)
    all_experiments_results_df.to_csv(all_results_path, index=False)
    all_results.append(eval_dict)

new_results_df = pd.DataFrame(all_results)
all_experiments_results_df = pd.read_csv(all_results_path)
summary_df = all_experiments_results_df[["experiment_name", "Useful_ratio", "3/3_ratio", "overall_punctuation", "time_mean", "time_std"]]
summary_df.to_csv(all_results_path.replace(".csv", "_summary.csv"), index=False)

In [ ]:
new_results_df.head()


In [31]:
len(all_experiments_results_df)

def _apply_extract_expname(row):
    settings = row["experiment_name"].split("_")
    if len(settings) < 5:
        return row
    row["model_name"] = settings[0]
    row["number_of_agents"] = int(settings[2].replace("n", "").strip())
    row["traits"] = settings[3].replace("T", "").strip()
    row["strategies"] = settings[4].replace("S", "").strip()
    row["round"] = int(len(row["strategies"]))
    return row

#all_experiments_results_df.apply(_apply_extract_expname, axis=1)


In [32]:
all_experiments_results_df.sort_values(by="overall_punctuation", ascending=False)[:30]

,experiment_name,Useful,Unhelpful,not_able_to_evaluate,Invalid,Useful_ratio,Unhelpful_ratio,not_able_to_evaluate_ratio,Invalid_ratio,3/3,2/3,1/3,0/3,3/3_ratio,2/3_ratio,1/3_ratio,0/3_ratio,overall_punctuation,time_mean,time_std
213,llama70b_social_n2_Tee_Sddd,438,61,40,19,0.78,0.11,0.07,0.03,99.0,62.0,17.0,8.0,0.53,0.33,0.09,0.04,0.784946,0.588126,2.454475
351,mistral24b_social_n3_Teeo_Sddr,434,67,39,18,0.78,0.12,0.07,0.03,101.0,52.0,27.0,6.0,0.54,0.28,0.15,0.03,0.777778,4.647916,7.665988
178,llama70b_social_n2_Tee_Sdd,432,64,44,18,0.77,0.11,0.08,0.03,96.0,60.0,24.0,6.0,0.52,0.32,0.13,0.03,0.774194,0.616250,2.866915
226,llama70b_social_n3_Teoo_Srdr,432,64,41,21,0.77,0.11,0.07,0.04,98.0,57.0,24.0,7.0,0.53,0.31,0.13,0.04,0.774194,3.187220,5.102949
202,llama70b_social_n2_Too_Sdrd,429,58,53,18,0.77,0.10,0.09,0.03,93.0,63.0,24.0,6.0,0.50,0.34,0.13,0.03,0.768817,2.343887,5.268516
236,llama70b_social_n3_Teeo_Sddd,428,64,48,18,0.77,0.11,0.09,0.03,99.0,57.0,17.0,13.0,0.53,0.31,0.09,0.07,0.767025,0.273109,3.076492
228,llama70b_social_n3_Tooo_Srdr,428,57,53,20,0.77,0.10,0.09,0.04,95.0,61.0,21.0,9.0,0.51,0.33,0.11,0.05,0.767025,0.140266,1.421920
212,llama70b_social_n2_Teo_Srdr,427,57,51,23,0.77,0.10,0.09,0.04,94.0,61.0,23.0,8.0,0.51,0.33,0.12,0.04,0.765233,4.324026,4.522981
173,llama70b_social_n3_Tooo_Sdr,427,69,43,19,0.77,0.12,0.08,0.03,98.0,55.0,23.0,10.0,0.53,0.30,0.12,0.05,0.765233,0.957643,2.566143
211,llama70b_social_n2_Teo_Srrd,427,55,51,25,0.77,0.10,0.09,0.04,102.0,48.0,25.0,11.0,0.55,0.26,0.13,0.06,0.765233,4.633286,4.653289


In [ ]:
summary_extended_df['model_name'].unique()

In [33]:
summary_df = summary_df.sort_values(by="overall_punctuation", ascending=False)
summary_extended_df = summary_df.apply(_apply_extract_expname, axis=1)
summary_extended_df['model_name'].unique()

summary_extended_df[summary_extended_df['model_name'] == "llama8b"][:10][["experiment_name", "Useful_ratio", "overall_punctuation", "3/3_ratio"]]

,experiment_name,Useful_ratio,overall_punctuation,3/3_ratio
124,llama8b_social_n3_Teee_Srdd,0.71,0.711470,0.40
57,llama8b_social_n3_Teee_Srd,0.70,0.700717,0.42
43,llama8b_social_n3_Teoo_Sd,0.69,0.693548,0.40
34,llama8b_social_n2_Tee_Sdd,0.70,0.693548,0.42
54,llama8b_social_n3_Teeo_Sdr,0.69,0.689964,0.39
51,llama8b_social_n3_Teoo_Srr,0.69,0.689964,0.39
60,llama8b_social_n3_Teeo_Srd,0.69,0.688172,0.41
88,llama8b_social_n2_Too_Sdrr,0.69,0.686380,0.38
121,llama8b_social_n3_Teee_Srrr,0.68,0.684588,0.39
13,llama8b_social_n1_Te_S,0.68,0.682796,0.00


In [ ]:
all_experiments_results_df.columns

In [ ]:
from glob import glob
import json

best_70b = "llama70b_social_n2_Tee_Sddd"
best_8b = "llama8b_social_n3_Teee_Srdd"
best_mistral = "mistral24b_social_n3_Teeo_Sddr"

best_models = {
    "70b": best_70b,
    "8b": best_8b,
    "mistral": best_mistral
}

# input_arg, round_answer_dict

from validator_agents.QuestionScorer import QuestionScorer

def load_last_state_and_validate(arg):
    agents = [x for x in arg['round_answer_dict'].keys() if x.startswith("agent")]
    arg_cqs = []
    for agent in agents:
        last_cq_list = arg['round_answer_dict'][agent][-1]["critical_question_list"]["critical_questions"]
        agent_cq = list(set([cq["critical_question"] for cq in last_cq_list]))
        arg_cqs.extend(agent_cq)

    arg_cqs = list(set(arg_cqs))
    original_arg = arg["input_arg"]


    # Instantiate Scorer
    scorer = QuestionScorer(LLM)

    # Sort questions, helpful first, unhelpful/invalid last
    sorted_questions = scorer.sort_questions(original_arg, arg_cqs)[:3]
    new_cqs = [{"cq": x, "id": i} for i, x in enumerate(sorted_questions)]
    return new_cqs

OUTPUT_FILES = "output/elbaff_experiment"
best_experiment_setting = best_8b
out_f = f"{OUTPUT_FILES}/output_{best_experiment_setting}.json"
new_out_f = f"{OUTPUT_FILES}/output_validation_{best_experiment_setting}.json"
arg_path = f"{OUTPUT_FILES}/final_states/{best_experiment_setting}_arg{{}}.json"

with open(out_f, "r")  as file:
    new_out = {}
    json_ = json.load(file)
    print(json_.keys())
    for arg_id_ in json_.keys():
        new_out[arg_id_] = json_[arg_id_] 
        new_out[arg_id_]['cqs'] = []
        # each arg
        with open(arg_path.format(arg_id_), "r")  as file:
            arg = json.load(file)
            new_out[arg_id_]['cqs'] = load_last_state_and_validate(arg)

    # SAVE NEW 
    with open(new_out_f, "w") as nof:
        json.dump(nof, file, indent=4)  # pretty print the json
arg

In [ ]:
5*70